In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time
import glob
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import csv
import cv2
import torchvision.transforms as transforms
import joblib

#libraries for yolo
from pytorchyolo.models import load_model
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
from pytorchyolo.utils.utils import non_max_suppression
from pytorchyolo.utils.loss import compute_loss

In [ ]:
from scripts.utils import detach_cpu, tensor_to_np_img, save_tensor_img, clone_detach, open_img_as_tensor, display_img

## Face Detector Models

In [ ]:
from scripts.face_detectors import MediaPipe, YuNet, YoloFace

In [ ]:
mp = MediaPipe()
yn = YuNet()
yf = YoloFace()

## Our Model

In [ ]:
from scripts.custom_mlp import *
our_model = joblib.load("model_dumps/penalty_1.1578947368421053_HSV_bbox_e_face_yf.pkl").best_estimator_

## YOLOFace with FGSM

In [ ]:
# Patterned after FGSM tutorial (https://pytorch.org/tutorials/beginner/fgsm_tutorial.html)
# Define what device we are using
print("CUDA Available: ", torch.cuda.is_available())
main_yf = YoloFace()
device, model = main_yf.device, main_yf.yf_face_detector


# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

## Image Feature Extraction

In [ ]:
from scripts import image_attributes

## Load Masks

## FGSM

In [ ]:
import scripts.fgsm as fgsm

In [ ]:
# Settings
image_attributes.save_color_images = False
image_attributes.save_lbp_images = False
image_attributes.save_gradient_images = False
fgsm_loss_target = "conf" # or "bbox"

In [ ]:
input_image = r"./images/1--Handshaking/1_Handshaking_Handshaking_1_51.jpg"

In [ ]:
masks[0].shape

In [ ]:
t1 = time.time()
data = open_img_as_tensor(input_image)
feats, grads, bboxes, masks = image_attributes.get_features(input_image)
feats = preprocess(feats.loc[:, our_model.feature_names_in_])
preds = our_model.predict(feats)
for e, data_grad, bbox, mask in zip(preds, grads, bboxes, masks):
    data = fgsm.fgsm_attack(data, e, data_grad, mask)
t2 = time.time()
print("Our regression model run time:", t2 - t1)
display_img(data)

In [ ]:
from scripts.facesegmentor import FaceSegementor
faceseg = FaceSegementor()

t1 = time.time()
data = open_img_as_tensor(input_image)
feats, grads, bboxes, masks = image_attributes.get_features(input_image, face_segmentor=faceseg)
feats = preprocess(feats.loc[:, our_model.feature_names_in_])
preds = our_model.predict(feats)
for e, data_grad, bbox, mask in zip(preds, grads, bboxes, masks):
    data = fgsm.fgsm_attack(data, e, data_grad, mask)
t2 = time.time()
print("Our regression model run time:", t2 - t1)
display_img(data)

In [ ]:
t1 = time.time()
data = open_img_as_tensor(input_image)
feats, grads, bboxes = get_features(input_image)
for data_grad, bbox in zip(grads, bboxes):
    bbox_mask = np.zeros(data.shape)
    bbox_mask[..., bbox[1]:bbox[3], bbox[0]:bbox[2]] = 1
    e = fgsm.binary_search(clone_detach(data), clone_detach(data_grad), yf, bbox_mask, bbox)
    data = fgsm.fgsm_attack(data, e, data_grad, bbox_mask)
t2 = time.time()
print("Binary Search run time:", t2 - t1)
display_img(data)

In [ ]:
yf.detect?

##### ___